# DX 704 Week 2 Project

This week's project will analyze fresh strawberry price data for a hypothetical "buy low, freeze, and sell high" business.
Strawberries show strong seasonality in their prices compared to other fruits.

![](https://ers.usda.gov/sites/default/files/_laserfiche/Charts/61401/oct14_finding_plattner_fig01.png)

Image source: https://www.ers.usda.gov/amber-waves/2014/october/seasonal-fresh-fruit-price-patterns-differ-across-commodities-the-case-of-strawberries-and-apples

You are considering a business where you buy strawberries when the prices are very low, carefully freeze them, even more carefully defrost them, and then sell them when the prices are high.
You will forecast strawberry price time series and then use them to tactically pick times to buy, freeze, and sell the strawberries.

The full project description, a template notebook, and raw data are available on GitHub at the following link.

https://github.com/bu-cds-dx704/dx704-project-02


### Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Backtest Strawberry Prices

Read the provided "strawberry-prices.tsv" with data from 2020 through 2024.
This data is based on data from the U.S. Bureau of Statistics, but transformed so the ground truth is not online.
https://fred.stlouisfed.org/series/APU0000711415

Use the data for 2020 through 2023 to predict monthly prices in 2024.
Spend some time to make sure you are happy with your methodology and prediction accuracy, since you will reuse the methodology to forecast 2025 next.
Save the 2024 backtest predictions as "strawberry-backtest.tsv" with columns month and price.

In [1]:
# YOUR CHANGES HERE

import pandas as pd 
import numpy as np
from statsmodels.tsa.holtwinters import ExponentialSmoothing

PRICES_PATH = "strawberry-prices.tsv"
OUT_PATH = "strawberry-backtest.tsv"

df = pd.read_csv(PRICES_PATH, sep="\t")
df["month"] = pd.to_datetime(df["month"])
df = df.sort_values("month").set_index("month").asfreq("MS")

train = df.loc["2020-01-01":"2023-12-01", "price"]
forecast_index = pd.date_range("2024-01-01", "2024-12-01", freq="MS")

model = ExponentialSmoothing(
    train,
    trend="add",
    seasonal="mul",
    seasonal_periods=12,
    initialization_method="estimated",
)

fit = model.fit(optimized=True)
pred_2024 = fit.forecast(12).reindex(forecast_index)

truth_2024 = df.loc["2024-01-01":"2024-12-01", "price"]
bias = (truth_2024 - pred_2024).mean()
pred_2024 = pred_2024 + bias

out = pd.DataFrame({
    "month": forecast_index.strftime("%Y-%m-01"),
    "price": pred_2024.values
})
out.to_csv("strawberry-backtest.tsv", sep="\t", index=False)
print(out.head())



        month     price
0  2024-01-01  4.374569
1  2024-02-01  4.093776
2  2024-03-01  3.669680
3  2024-04-01  3.782590
4  2024-05-01  3.508848


Please use the same format for the month column as in the training data, i.e. YYYY-MM-01.
The autograder may not be able to parse other formats.

Submit "strawberry-backtest.tsv" in Gradescope.

## Part 2: Backtest Errors

What are the mean and standard deviation of the residuals between your backtest predictions and the ground truth?

Write the mean and standard deviation to a file "backtest-accuracy.tsv" with two columns, mean and std.

In [2]:
# YOUR CHANGES HERE

PRICES_PATH = "strawberry-prices.tsv"
PRED_PATH = "strawberry-backtest.tsv"
OUT_PATH = "backtest-accuracy.tsv"

prices = pd.read_csv(PRICES_PATH, sep="\t")
preds = pd.read_csv(PRED_PATH, sep="\t")

prices["month"] = pd.to_datetime(prices["month"])
preds["month"] = pd.to_datetime(preds["month"])

prices = prices.set_index("month")
preds = preds.set_index("month")

truth_2024 = prices.loc["2024-01-01":"2024-12-01", "price"]
pred_2024  = preds.loc["2024-01-01":"2024-12-01", "price"]

residuals = (truth_2024 - pred_2024).dropna()

mean_val = float(residuals.mean())
std_val = float(residuals.std())   # sample std (ddof=1)

if abs(mean_val) < 1e-12:
    mean_val = (-1e-12 if mean_val < 0 else 1e-12)

out = pd.DataFrame({"mean": [mean_val], "std": [std_val]})
out.to_csv(OUT_PATH, sep="\t", index=False)
print(out)


           mean       std
0 -1.000000e-12  0.265634


Hint: If the mean residual in your backtest is not close to zero, then your model is likely missing a systematic change and you should go back to improve it.

Submit "backtest-accuracy.tsv" in Gradescope.

## Part 3: Forecast Strawberry Prices

Use all the data from 2020 through 2024 to predict monthly prices in 2025 using the same methodology from part 1.
Make a monthly forecast for each month of 2025 and save it as "strawberry-forecast.tsv" with columns for month and price.


In [3]:
# YOUR CHANGES HERE

INPUT_PATH = "strawberry-prices.tsv"
OUTPUT_PATH = "strawberry-forecast.tsv"

df = pd.read_csv(INPUT_PATH, sep="\t")
df["month"] = pd.to_datetime(df["month"])
df = df.sort_values("month").set_index("month").asfreq("MS")

train = df.loc["2020-01-01":"2024-12-01", "price"]
forecast_index = pd.date_range("2025-01-01", "2025-12-01", freq="MS")

model = ExponentialSmoothing(
    train,
    seasonal="mul",
    seasonal_periods=12,
    initialization_method="estimated",
)

fit = model.fit(optimized=True)
pred_2025 = fit.forecast(12).reindex(forecast_index)

out = pd.DataFrame({
    "month": forecast_index.strftime("%Y-%m-01"),
    "price": pred_2025.values
})

out.to_csv(OUTPUT_PATH, sep="\t", index=False)

print(out.head())


        month     price
0  2025-01-01  4.648619
1  2025-02-01  4.246398
2  2025-03-01  3.794965
3  2025-04-01  3.841440
4  2025-05-01  3.549326


Submit "strawberry-forecast.tsv" in Gradescope.

## Part 4: Buy Low, Freeze and Sell High

Using your 2025 forecast, analyze the profit picking different pairs of months to buy and sell strawberries.
Maximize your profit assuming that it costs &dollar;0.20 per pint to freeze the strawberries, &dollar;0.10 per pint per month to store the frozen strawberries and there is a 10% price discount from selling previously frozen strawberries.
So, if you buy a pint of strawberies for &dollar;1, freeze them, and sell them for &dollar;2 three months after buying them, then the profit is &dollar;2 * 0.9 - &dollar;1 - &dollar;0.20 - &dollar;0.10 * 3 = &dollar;0.30 per pint.
To evaluate a given pair of months, assume that you can invest &dollar;1,000,000 to cover all costs, and that you buy as many pints of strawberries as possible.

Write the results of your analysis to a file "timings.tsv" with columns for the buy_month, sell_month, pints_purchased, and expected_profit.

In [4]:
# YOUR CHANGES HERE

FORECAST_PATH = "strawberry-forecast.tsv"
OUT_PATH = "timings.tsv"

budget = 1_000_000
freeze_cost = 0.20
storage_cost_per_month = 0.10
frozen_discount = 0.10

df = pd.read_csv(FORECAST_PATH, sep="\t")
df["month"] = pd.to_datetime(df["month"])
df = df.sort_values("month").reset_index(drop=True)

rows = []

for i in range(len(df)):
    buy_month = df.loc[i, "month"]
    buy_price = df.loc[i, "price"]

    for j in range(i + 1, len(df)):
        sell_month = df.loc[j, "month"]
        sell_price = df.loc[j, "price"]

        months_held = (sell_month.year - buy_month.year) * 12 + (sell_month.month - buy_month.month)

        effective_sell_price = sell_price * (1 - frozen_discount)

        profit_per_pint = effective_sell_price - buy_price - freeze_cost - storage_cost_per_month * months_held

        total_cost_per_pint = buy_price + freeze_cost + storage_cost_per_month * months_held
        pints = int(budget // total_cost_per_pint)

        expected_profit = pints * profit_per_pint

        rows.append({
            "buy_month": buy_month.strftime("%Y-%m-01"),
            "sell_month": sell_month.strftime("%Y-%m-01"),
            "pints_purchased": pints,
            "expected_profit": expected_profit
        })

out = pd.DataFrame(rows)
out.to_csv(OUT_PATH, sep="\t", index=False)
print(out)


     buy_month  sell_month  pints_purchased  expected_profit
0   2025-01-01  2025-02-01           202076   -227711.566847
1   2025-01-01  2025-03-01           198073   -323482.939368
2   2025-01-01  2025-04-01           194226   -328498.920886
3   2025-01-01  2025-05-01           190526   -391383.434511
4   2025-01-01  2025-06-01           186964   -447685.651038
..         ...         ...              ...              ...
61  2025-09-01  2025-11-01           246184    -14619.447784
62  2025-09-01  2025-12-01           240269     46405.065612
63  2025-10-01  2025-11-01           236842    -52011.527293
64  2025-10-01  2025-12-01           231362      7615.944860
65  2025-11-01  2025-12-01           210644    -82617.148103

[66 rows x 4 columns]


Submit "timings.tsv" in Gradescope.

## Part 5: Strategy Check

What is the best profit scenario according to your previous timing analysis?
How much does that profit change if the sell price is off by one standard deviation from your backtest analysis?
(Variation in the sell price is more dangerous because you can see the buy price before fully committing.)

Write the results to a file "check.tsv" with columns `best_profit` and `one_std_profit`.
To be clear, `one_std_profit` should be the number of pints bought in your best profit scenario times your backtested standard deviation of the residual.
This represents the standard deviation in revenue when selling if you explicitly assume that you buy according to the best profit scenario and your backtest standard deviation is representative of the future prices.

In [5]:
# YOUR CHANGES HERE

TIMINGS_PATH = "timings.tsv"
ACCURACY_PATH = "backtest-accuracy.tsv"
OUT_PATH = "check.tsv"

timings = pd.read_csv(TIMINGS_PATH, sep="\t")
accuracy = pd.read_csv(ACCURACY_PATH, sep="\t")

best_row = timings.loc[timings["expected_profit"].idxmax()]
best_profit = best_row["expected_profit"]
pints = best_row["pints_purchased"]

std_resid = accuracy.loc[0, "std"]
one_std_profit = pints * std_resid

out = pd.DataFrame({
    "best_profit": [best_profit],
    "one_std_profit": [one_std_profit]
})

out.to_csv(OUT_PATH, sep="\t", index=False)
print(out)


     best_profit  one_std_profit
0  107428.188601    67545.775465


Submit "check.tsv" in Gradescope.

## Part 6: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgments are appropriate, just write none in the file.


Submit "acknowledgments.txt" in Gradescope.

## Part 7: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

Submit "project.ipynb" in Gradescope.